# 개요

데이터 분석 및 전처리 코드


10월 19일
AIDU 대회 예선 코드


# 1. 파일 업로드 및 기본 세팅

In [ ]:

# AIDU Library Import
from aicentro.session import Session
from aicentro.framework.keras import Keras as AiduFrm
aidu_session = Session(verify=False)
aidu_framework = AiduFrm(session=aidu_session)

# 데이터 파일 경로
data_path = aidu_framework.config.data_dir

import subprocess
import sys
import numpy as np

import warnings
# 불필요한 경고 출력을 방지합니다.
warnings.filterwarnings('ignore')

#필요한 모델 다운로드
subprocess.call([sys.executable,"-m","pip","install","xgboost"])
subprocess.call([sys.executable,"-m","pip","install","seaborn"])
subprocess.call([sys.executable,"-m","pip","install","lightgbm"])
subprocess.call([sys.executable,"-m","pip","install","catboost"])

In [ ]:
#파일 명

FILE_NAME = 'jaeho_1012_set'

In [ ]:

#Train file 데이터 디렉토리
train_file_dir = data_path + "/episode1_train_-99.csv"
train_ob_file_dir = data_path + "/episode1_train_ob.csv"

#Test file 데이터 디렉토리
test_file_dir = data_path + "/episode1_test_-99.csv"
test_ob_file_dir = data_path + "/episode1_test_ob.csv"


In [ ]:
import pandas as pd

#원본 데이터
train_raw_df = pd.read_csv(train_file_dir)
train_ob_raw_df = pd.read_csv(train_ob_file_dir)

#테스트 원본 데이터 
test_raw_df = pd.read_csv(test_file_dir)
test_ob_raw_df = pd.read_csv(test_ob_file_dir)



In [ ]:
#트레인 데이터 구조
print(train_raw_df.shape)
print(train_ob_raw_df.shape)

#테스트 데이터셋 구조
print(test_raw_df.shape)
print(test_ob_raw_df.shape)

In [ ]:
#결측치 확인
print(train_raw_df.isnull().sum().sum())
print(train_ob_raw_df.isnull().sum().sum())

print(test_raw_df.isnull().sum().sum())
print(test_ob_raw_df.isnull().sum().sum())

In [ ]:
#train ob데이터와 합침 (id를 기준으로)
train_raw_df = pd.merge(train_raw_df, train_ob_raw_df, how = 'outer', on='id')
train_raw_df = train_raw_df.fillna(0)

#test_ob데이터와 합침 (id를 기준으로)
test_raw_df = pd.merge(test_raw_df, test_ob_raw_df, how = 'outer', on='id')
test_raw_df = test_raw_df.fillna(0)


print(train_raw_df.shape)
print(test_raw_df.shape)


### ★★★★★★★여기부터 시작 전처리 및 EDA 시작★★★★★★★★

In [ ]:
# TEST 데이터를 Train 데이터 밑에 합침
merge_raw_df = pd.concat([train_raw_df, test_raw_df])

In [ ]:
#전처리 후 모델학습을 위한 데이터
merge_df = pd.DataFrame()
train_df = pd.DataFrame()
test_df = pd.DataFrame()

### 전체 컬럼

In [ ]:
all_col = merge_raw_df.columns
all_col

## 1. 데이터 전처리

### 1.1 N이상수는 N으로 처리하기

In [ ]:
#n 이상을 n 으로 합치는 컬럼

#n이상 숫자를 n로 대치
case_oulier_n_col = [
                   'voc_cnt_n',
                     'accum_usst_day_num_divide_10',
                     'tm_cant_call_cnt','tm_etc_cnt',
                     'tm_npay_cnt','tm_use_stop_cnt',
                     'sm_npay_cnt','sm_thismonth_cnt', 'recnt_icg_date_from_this_year'
                     ]


def oulier_up_n_num(input_df, col_name, n):
    '''
    n보다 큰 수는 n으로 처리하는 함수
    param:
    input_df: 인풋함수:
    col_name: 컬럼명
    n : n 이상 처리할 숫자
    '''
    
    fun_df = pd.DataFrame()
      
    def fun(x):
        if x > n:
            return n
        else:
            return x
        
    fun_df = input_df[col_name].apply(lambda x : fun(x) )
    
    return fun_df


In [ ]:
# n보다 큰 수 n로 만들기
for index in case_oulier_n_col:
    merge_df[index] = oulier_up_n_num(merge_raw_df, index, 6)


### 1.2 0과 0이 아닌것으로 나누기

In [ ]:
#n 이상을 n 으로 합치는 컬럼
case_alpha_zero_or_not_col = [
#                              'voc_inqr_cascnt_n',
#                                 'voc_tot_occ_cascnt_n',
#                                 'rmonth_qat_chage_voc_cascnt',
#                                 'accum_usst_day_num_divide_10',
#                                 'tm_cant_call_cnt',
#                                 'tm_etc_cnt',
#                                 'tm_use_stop_cnt',
#                                 'sm_use_stop_cnt',
#                                 'sm_use_stop_cnt'
                            ]

def zero_or_not(input_df, col_name):
    '''
    0 / 0 아닌수로 대치하여 YN 변경
    param:
    
    input_df: 인풋함수:
    col_name: 컬럼명
    '''
    
    fun_df = pd.DataFrame()
    
    def fun(x):
        if x ==0:
            return 'N'
        else:
            return "Y"
        
    fun_df = input_df[col_name].apply(lambda x : fun(x) )
    
    return fun_df


In [ ]:
# n보다 큰 수 n로 만들기
for index in case_alpha_zero_or_not_col:
    merge_df[index] = zero_or_not(merge_raw_df, index)


### 1.3 이상치를 IQR highest로 교체


In [ ]:
def get_outlier(df=None, column=None, weight=1.5):
    '''
    outlier 추출하여 index로 리턴
    
    param 
    df: 추출하고자 하는 데이터 프레임 전체
    column : 이상치 확인 하려는 컬럼
    weight : 가중치
    
    '''
  # target 값과 상관관계가 높은 열을 우선적으로 진행
    quantile_25 = np.percentile(df[column].values, 25)
    quantile_75 = np.percentile(df[column].values, 75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight
  
    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight
    
    iqr = highest

    outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index

    return iqr, outlier_idx

In [ ]:
def outlier_up_iqr_num(input_df, col_name, iqr):
    '''
    iqr보다 큰 수는 iqr로 처리하는 함수
    param:
    input_df : 인풋함수
    col_name : 컬럼명
    n : n 이상 처리할 숫자
    '''
    fun_df = pd.DataFrame()
    
    def fun(x):
        if x > iqr:
            return iqr
        else : 
            return x
        
    fun_df = input_df[col_name].apply(lambda x : fun(x))
    
    return fun_df

In [ ]:
case_iqr_col =[
  'r3m_avg_bill_amt', 'r3m_mphon_avg_arpu_amt', 'r3m_iptv_avg_arpu_amt', 
       'r3m_inet_avg_arpu_amt','r6m_mphon_avg_arpu_amt', 'r6m_iptv_avg_arpu_amt',
       'r6m_inet_avg_arpu_amt',
              ]

#이상치 값을 IQR highest로 대치하여 정상화
for index in case_iqr_col:
    # 함수 사용해서 이상치 값 삭제
    iqr, oulier_idx = get_outlier(df=merge_raw_df, column=index, weight=2.3)
    merge_df[index] = outlier_up_iqr_num(merge_raw_df, index, iqr)

### 1.4 라벨인코딩

In [ ]:
#라벨 인코딩할 컬럼
case_label_encoding_col=[
                        'label_payment_yn',
                        'inet_sbsc_yn',
                        'soip_sbsc_yn',
                        'iptv_sbsc_yn',
                        'pstn_sbsc_yn',
                        'pstn_sbsc_yn',
                        'dt_stop_yn',
                        'mphon_comb_yn',
                        'mphon_sbsc_yn',
                        'inet_comb_yn',
                        'iptv_comb_yn',
                        'npay_yn',
                        'hndset_rmnd_insl_mons_yn',
                        'r6m_inet_pssn_comb_yn',
                        'r6m_iptv_pssn_comb_yn',
                        'kids_wrlin_adtn_svc_sbsc_yn',
                        'r6m_iptv_first_ppv_use_yn',
                        'bill_rmny_npay_tms_itg_cd',
                        'r3m_avg_bill_amt_100000_over_yn',
                        'r3m_wless_data_use_qnt_100000_over_yn',
                        'insur_prod_sbsc_yn',
    

                        ]


def labelencoding( input_df, col):
    from sklearn import preprocessing
    le=preprocessing.LabelEncoder()
    
    #라벨링 전처리 후 데이터
    le_eda_df = pd.DataFrame()
    
        
    #전처리하기 전에 해당 컬럼 전처리 후 컬럼에 추가
    le_eda_df[col] = input_df[col]

    if col =='label_payment_yn':
        le_eda_df = le_eda_df.fillna('N')
        
    # 라벨 인코딩
    le.fit(le_eda_df[col])

    # train-set, test-set 둘다 Transform
    le_eda_df[col] = le.transform(le_eda_df[col])
    
    return le_eda_df[col]

In [ ]:
# 라벨 인코딩 
for index in case_label_encoding_col:
    merge_df[index] = labelencoding(merge_raw_df, index)

### 1.5 원핫인코딩

In [ ]:
case_onehot_encoding_col = [
   'pay_way_nm',
 'acrnd_dsp_meth_nm',
 'anals_2_prod_level_nm',
 'bprod_lctg_nm',
 'wless_terml_gun_div_nm',
 'wless_terml_gun_div_type'
                            ]

def onehot_encoding(input_df, col):
    
    #카테고리 갯수로 데이터프레임 만듬
    index_value_df =pd.DataFrame ()
    index_value_df['cat'] = list(input_df[col].value_counts().index)                           
    index_value_df['value']= list(input_df[col].value_counts())
    
    fun_df = pd.DataFrame()
    
    fun_df = pd.get_dummies(input_df[col], prefix=col)
    
    return fun_df


    


In [ ]:
for index in case_onehot_encoding_col:
    
    onehot_df =onehot_encoding(merge_raw_df,index )
       
    merge_df[onehot_df.columns] = onehot_df

In [ ]:
case_onehot_encoding_col

### 1.6 전처리 안 해준 데이터 추가

In [ ]:
#전체 컬럼에서, 전처리했던 컬럼 제거하여 전처리 안한 컬럼 추출
add_raw_col=list(set(merge_raw_df.columns)-set(merge_df.columns))

In [ ]:
#전처리 안한 컬럼 추가
for index in add_raw_col:
    merge_df[add_raw_col] = merge_raw_df[add_raw_col]

In [ ]:
add_raw_col

### 1.7 정규화

In [ ]:
for index in merge_df.columns:
    if merge_df[index].dtype =='O':
        merge_df[index] = labelencoding(merge_df,index)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

#정규화 할 컬럼 , 나머지는 roboster 정규화 진행
nomalize_col=[ 'svc_use_mons_num_divide_10','sm_athrt_trmn_cnt', 'sm_term_stop_cnt', 'tm_cant_collect_cnt','efct_iptv_sbsc_cascnt', 'hndset_rmnd_insl_mons_num',   'npay_tms_cnt',    'efct_pstn_sbsc_cascnt']

scaler = RobustScaler()
#로보스터 할 데이터
roboster_col = list(set(merge_df.columns)-set(nomalize_col))
merge_df[roboster_col] = scaler.fit_transform( merge_df[roboster_col])


#정규화 할 데이터
scaler = MinMaxScaler()
#nomalize_col = merge_df.columns
merge_df[nomalize_col] = scaler.fit_transform( merge_df[nomalize_col])




### 1.8 변수 중요도 낮은 컬럼 제거

In [ ]:
drop_col = [ 'id','voc_sbsc_inqr_cascnt_n',
            'voc_dscnt_cascnt_n','voc_icnv_cascnt_n',
            'tm_athrt_trmn_cnt','tm_handling_complaint_cnt',
            'tm_promise_pay_cnt', 'tm_term_stop_cnt']

# drop_list=['sm_term_stop_cnt', 'voc_icnv_cascnt_n', 'tm_cant_collect_cnt']

merge_df.drop( drop_col,axis=1,inplace=True )


### 1.9 모델 전처리 출력

어떤 전처리를 했는지 출력해주는 부분

In [ ]:
description = data_path + "/episdode1_train_description.csv"
description_ob = data_path + "/episdode1_train_ob_description.csv"

description_df=pd.read_csv(description)
description_ob_df=pd.read_csv(description_ob)

description_df = pd.concat([description_df,description_ob_df], ignore_index=True)


eda_column_df = pd.DataFrame()
eda_column_df['기준컬럼'] = merge_raw_df.columns


for row in range(eda_column_df.shape[0]):
    str_eng= eda_column_df.loc[row,'기준컬럼']
    for i in description_df['컬럼 영문명']:
        if i in str_eng:
            eda_column_df.loc[row,'컬럼 한글명'] = description_df.loc[description_df['컬럼 영문명']==i, '컬럼 한글명'].values[0]

# 전체 - 으로 세팅            
eda_column_df['전처리결과'] = '-'

In [ ]:


def eda_check (eda_fun_col,check_name):

    for i in eda_fun_col:
        try:
            for row in range(eda_column_df.shape[0]):
                str_name = eda_column_df.loc[row,'기준컬럼']
                if str_name in i:
                    row_name = str_name
            data_st =eda_column_df.loc[eda_column_df['기준컬럼']==row_name,'전처리결과'].values[0]

            if data_st == '-':
                eda_column_df.loc[eda_column_df['기준컬럼']==row_name,'전처리결과']= check_name
            else:
                if check_name not in data_st:
                    eda_column_df.loc[eda_column_df['기준컬럼']==row_name,'전처리결과']= data_st+"/ " +check_name
        except:
            pass




In [ ]:
eda_check(drop_col,'삭제' )
eda_check(case_oulier_n_col,'nton' )
eda_check(case_alpha_zero_or_not_col,'0roNot' )
eda_check(case_iqr_col,'IQR' )
eda_check(case_onehot_encoding_col,'원핫인코딩' )
eda_check(add_raw_col,'미처리' )
eda_check(nomalize_col,'정규화' )
eda_check(roboster_col,'로보스터' )


In [ ]:
# pd.set_option('display.max_rows', None)
eda_column_df


In [ ]:
#전처리 한 결과 출력해주는 부분 

eda_file_name = data_path + "/fit/eda_result_check_"+FILE_NAME+".csv"
eda_column_df.to_csv(eda_file_name, index=False )
print(eda_file_name,"파일이 출력되었습니다.")

### 1.10 상관값 높은 컬럼 제거

In [ ]:
from sklearn.model_selection import train_test_split

train_df = merge_df.iloc[:train_raw_df.shape[0]]
test_df = merge_df.iloc[train_raw_df.shape[0]:]

#상관관계 비교를 위해 
x_train, x_valid, y_train, y_valid = train_test_split(train_df.drop(['label_payment_yn'],axis=1), train_df['label_payment_yn'],random_state=21)


In [ ]:
#상관관계 비교하여 드롭할 컬럼 추출
print(x_train.shape)
corr_matrix = x_train.corr()
theshold =0.95

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))

#상관값 높은 컬럼
to_drop = [column for column in upper.columns if any(abs(upper[column]) >theshold)]

merge_df.drop(to_drop,axis=1,inplace=True)

### 1.11 PCA 차원 축소
여긴 이어지지는 않고 필요하면 원하는 컬럼만 차원 축소

In [ ]:
from sklearn.decomposition import PCA

compoent = int(merge_df.shape[1]*0.8)

pca = PCA(n_components=compoent) # 주성분을 몇개로 할지 결정


printcipalComponents = pca.fit_transform(merge_df.drop(['label_payment_yn'],axis=1))

principalDf = pd.DataFrame(data=printcipalComponents)



# 2. 데이터 분리 및 모델 학습 준비

In [ ]:
from sklearn.model_selection import train_test_split

train_df = merge_df.iloc[:train_raw_df.shape[0]]
test_df = merge_df.iloc[train_raw_df.shape[0]:]

print(train_df.shape)
print(test_df.shape)

In [ ]:
#전처리 완료한 파일로 출력하여 모델학습으로 이어짐

test_file_name =data_path + "/pre_data/test_preprocessing_"+FILE_NAME+".csv"
    
train_file_name = data_path + "/pre_data/train_preprocessing_"+FILE_NAME+".csv"
    
# 학습할 부분과 테스트 할 부분 나눔
test_df.drop(['label_payment_yn'],axis=1, inplace=True)
test_df.to_csv(test_file_name, index=False)
print(test_file_name,": 파일이 출력되었습니다.")

train_df.to_csv(train_file_name, index=False)
print(train_file_name, ": 파일이 출력되었습니다.")    
    
